In [6]:
import numpy as np
from scipy.io.wavfile import write
import os
from tqdm import tqdm

count_wav_files_in_folder() function count how many files in the certain folder

In [7]:
def count_wav_files_in_folder(folder_path):
    wav_count = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".wav"):
            wav_count += 1
    return wav_count

These cell include every changable parameters in the Signal Generation part:

1. folder_1 and folder_2 are the signals collected from LoRa Transceiver by HackRF One.
2. We assume the amount of the signals are fixed since the VRAM limitaion. Therefore, the generated signals should be (Fixed numbers) minus (All LoRa signal).
3. Other parameters are affected to the generated signals. These parameters make them closed to the LoRa signal while maintain differences with each of them.


In [8]:
folder_1 = '/Users/yukewei/SP-testing/HackRFOneSignal/AttackerSignal'
folder_2 = '/Users/yukewei/SP-testing/HackRFOneSignal/KeyFobSignal'
output_folder = "/Users/yukewei/SP-testing/Generated Signal/Non processed Signal/"

num_files = 10000 - count_wav_files_in_folder(folder_1) - count_wav_files_in_folder(folder_2)  # Number of .wav files to generate
duration = 0.3  # Total duration of the .wav file (300ms)
actual_duration = 0.22  # Actual signal duration (220ms)
sample_rate = 2000000
fixed_frequency = 433000000  # 433 MHz
frequency_range = 0.0625e6  # ±0.0625 MHz

total_duration = 0.3  # Total duration of the .wav file (300ms)
allowed_start_range = (0.0, 0.08)  # Allowed start time range (0ms to 80ms) like  LoRa signals

amplitude_range = (0.5, 10)
phase_range = (0, 2 * np.pi)


noise_amplitude = 100  

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [10]:
with tqdm(total=num_files, desc="Generating .wav files") as pbar:
    for i in range(num_files):
        # Generate random amplitudes, phases, and start times
        amplitude = np.random.uniform(amplitude_range[0], amplitude_range[1])
        phase_offset = np.random.uniform(phase_range[0], phase_range[1])
        start_time = np.random.uniform(allowed_start_range[0], allowed_start_range[1])

        # Calculate the end time based on the allowed start time and signal duration
        end_time = start_time + actual_duration

        # Generate the actual signal
        t_signal = np.linspace(start_time, end_time, int(sample_rate * actual_duration), endpoint=False)

        frequency_offset = np.random.uniform(-frequency_range, frequency_range)  # Random frequency offset

        # Create a signal with a range of frequencies
        actual_frequencies = fixed_frequency + frequency_offset
        signal = amplitude * np.sin(2 * np.pi * actual_frequencies * t_signal + phase_offset)

        # Generate random noise to make the signal less cyclical
        noise = noise_amplitude * np.random.randn(len(signal))
        signal += noise

        # Generate random silent space before and after the signal.
        # The signal cannot be splitted by silent space
        pre_silence_duration = start_time
        post_silence_duration = total_duration - end_time

        pre_silence_samples = int(pre_silence_duration * sample_rate)
        post_silence_samples = int(post_silence_duration * sample_rate)

        
        
        if pre_silence_samples > 0:
            pre_silence = np.zeros(pre_silence_samples)
            signal = np.concatenate((pre_silence, signal))
            

        if post_silence_samples > 0:
            post_silence = np.zeros(post_silence_samples)
            signal = np.concatenate((signal, post_silence))
            

        # Ensure the final signal duration is 300ms
        if len(signal) > sample_rate * total_duration:
            signal = signal[:int(sample_rate * total_duration)]
        elif len(signal) < sample_rate * total_duration:
            # Add zeros to the end of the signal to make it 300ms
            zeros_to_add = int(sample_rate * total_duration) - len(signal)
            signal = np.concatenate((signal, np.zeros(zeros_to_add)))

        # Convert signal to 16-bit PCM format
        signal_int16 = np.int16(signal * 32767)

        # Define the filename based on the variations
        filename = f"{int(actual_frequencies / 1e6)}MHz_{sample_rate}Hz_{i}.wav"
        filepath = os.path.join(output_folder, filename)

        # Save the signal as a .wav file
        write(filepath, sample_rate, signal_int16)
        pbar.update(1)

print("Generation of .wav files with variations and randomly added noise complete.")


Generating .wav files:   0%|          | 0/5918 [00:00<?, ?it/s]

Generating .wav files: 100%|██████████| 5918/5918 [01:23<00:00, 70.95it/s]

Generation of .wav files with variations and randomly added noise complete.
